In [1]:
from traitlets import Unicode, Bool, validate, TraitError
from ipywidgets import DOMWidget, register

In [2]:
@register
class Email(DOMWidget):
    _view_name = Unicode('EmailView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes: ipython traitlets
    value = Unicode('example@example.com', help="The email value.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes.").tag(sync=True)
    
    # Basic validator for the email value
    @validate('value')
    def _valid_value(self, proposal):
        if proposal['value'].count("@") != 1:
            raise TraitError('Invalid email value: it must contain an "@" character')
        if proposal['value'].count(".") == 0:
            raise TraitError('Invalid email value: it must contain at least one "." character')
        return proposal['value']

In [3]:
%%javascript
require.undef('email_widget');
define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var EmailView = widgets.DOMWidgetView.extend({
        // Render the view.
        render: function() {
            this.email_input = document.createElement('input');
            this.email_input.type = 'email';
            this.email_input.value = this.model.get('value');
            this.email_input.disabled = this.model.get('disabled');
            
            // The `el` property is the DOM element associated with the view
            this.el.appendChild(this.email_input);
            
            // Python -> JavaScript update
            this.model.on('change:value', this.value_changed, this);
            this.model.on('change:disabled', this.disabled_changed, this);
            
            // JavaScript -> Python update
            this.email_input.onchange = this.input_changed.bind(this);
        },
        
        value_changed: function() {
            this.email_input.value = this.model.get('value');
        }, 
        
        disabled_changed: function() {
            this.email_input.disabled = this.model.get('disabled');
        },
        
        input_changed: function() {
            this.model.set('value', this.email_input.value);
            this.model.save_changes();
        }
    });
    return {
        EmailView: EmailView
    };
});

<IPython.core.display.Javascript object>

In [4]:
email = Email(value='dahuang@domain.com', disabled=False)
email

Email(value='dahuang@domain.com')